# `ganda` User Guide

This user guide is built in an interactive `bash` Jupyter Notebook.  If you've got `ganda` [installed](../README.md#installation) and in your `PATH` you can run the same commands.

In [19]:
# Setup  
# create a scratch directory for input/output files
mkdir -p scratch

# ensure that the ganda executable is in your PATH
which ganda >/dev/null && echo "ganda found in PATH" || echo "ganda not found in PATH"

ganda found in PATH


# `ganda` Usage

In [20]:
ganda help

NAME:
   ganda - make http requests in parallel

USAGE:
   <urls/requests on stdout> | ganda [options]

VERSION:
   dev none unknown

DESCRIPTION:
   Pipe urls to ganda over stdout for it to make http requests to each url in parallel.

AUTHOR:
   Ted Naleid <contact@naleid.com>

COMMANDS:
   echoserver  Starts an echo server, --port <port> to override the default port of 8080
   help, h     Shows a list of commands or help for one command

GLOBAL OPTIONS:
   --base-retry-millis value                              the base number of milliseconds to wait before retrying a request, exponential backoff is used for retries (default: 1000)
   --response-body value, -B value                        transforms the body of the response. Values: 'raw' (unchanged), 'base64', 'discard' (don't emit body), 'escaped' (JSON escaped string), 'sha256' (default: raw)
   --connect-timeout-millis value                         number of milliseconds to wait for a connection to be established before timeout (d

# `ganda` Basics


`ganda` makes HTTP requests, similar to `curl`, just pipe it an URL on stdin and it will make a `GET` request and echo the body of the response on stdout.  The status code of the URL will be sent to stderr.

We'll use [httpbin.org](http://httpbin.org) for the first few requests.  It returns a JSON representation of the request in the body of the response.

In [21]:
echo "http://httpbin.org/anything/1" | ganda 

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept-Encoding": "gzip", 
    "Host": "httpbin.org", 
    "User-Agent": "Go-http-client/1.1", 
    "X-Amzn-Trace-Id": "Root=1-66a6f801-28ffd07922c5ad2a20aee77f"
  }, 
  "json": null, 
  "method": "GET", 
  "origin": "173.16.32.166", 
  "url": "http://httpbin.org/anything/1"
}
Response: 200 http://httpbin.org/anything/1



You can pipe multiple URLs to `ganda`.  It happily lives in the middle of shell pipes for making requests.

Here we make 3 requests to `/anything/1`, `/anything/2`, and `/anything/3` and pipe them to `jq` where we grab just the `method` and `url` properties from the response.

We've also added the `-s` (silent) flag to `ganda` to suppress the stderr output that shows the url and response codes.

In [22]:
seq 3 |\
  awk '{printf "http://httpbin.org/anything/%s\n", $1}' |\
  ganda -s |\
  jq -c '{method, url}'

{"method":"GET","url":"http://httpbin.org/anything/1"}
{"method":"GET","url":"http://httpbin.org/anything/2"}
{"method":"GET","url":"http://httpbin.org/anything/3"}


## JSON Output

`ganda` uses the `-J` flag for JSON output.  This emits JSON with the response as the `"body"` field and includes other details about the request:

In [23]:
echo "http://httpbin.org/anything/1" |\
  ganda -s -J |\
  jq '.'

{
  "url": "http://httpbin.org/anything/1",
  "code": 200,
  "body": {
    "args": {},
    "data": "",
    "files": {},
    "form": {},
    "headers": {
      "Accept-Encoding": "gzip",
      "Host": "httpbin.org",
      "User-Agent": "Go-http-client/1.1",
      "X-Amzn-Trace-Id": "Root=1-66a6f805-54a88c0528124687501029a3"
    },
    "json": null,
    "method": "GET",
    "origin": "173.16.32.166",
    "url": "http://httpbin.org/anything/1"
  }
}


The body of the response is assumed to be JSON as a default.  This emits the `raw` response bytes after the `"body"` property.  If the response isn't JSON, you've got a few options for escaping/encoding the response using the `-B/--response-body <value>` flag:

1. `raw` - the default, shown above
2. `base64` - encode the bytes as a `base64` string, useful for binary content.
3. `discard` - drop the bytes and set the body to `null` 
4. `escaped` - escape the JSON and emit the value as a String
5. `sha256` - calculate the sha256 value of the body, useful for checking if the response has changed

In [24]:
# base64 encode the response body
echo "http://httpbin.org/anything/1" |\
  ganda -s -J -B base64 |\
  jq '.'

{
  "url": "http://httpbin.org/anything/1",
  "code": 200,
  "body": "ewogICJhcmdzIjoge30sIAogICJkYXRhIjogIiIsIAogICJmaWxlcyI6IHt9LCAKICAiZm9ybSI6IHt9LCAKICAiaGVhZGVycyI6IHsKICAgICJBY2NlcHQtRW5jb2RpbmciOiAiZ3ppcCIsIAogICAgIkhvc3QiOiAiaHR0cGJpbi5vcmciLCAKICAgICJVc2VyLUFnZW50IjogIkdvLWh0dHAtY2xpZW50LzEuMSIsIAogICAgIlgtQW16bi1UcmFjZS1JZCI6ICJSb290PTEtNjZhNmY4MDctMDZiYmFiOTk3NTMzZGE3MjQyZmU5ZDI1IgogIH0sIAogICJqc29uIjogbnVsbCwgCiAgIm1ldGhvZCI6ICJHRVQiLCAKICAib3JpZ2luIjogIjE3My4xNi4zMi4xNjYiLCAKICAidXJsIjogImh0dHA6Ly9odHRwYmluLm9yZy9hbnl0aGluZy8xIgp9Cg=="
}


In [25]:
# discard the response body
echo "http://httpbin.org/anything/1" |\
  ganda -s -J -B discard |\
  jq '.'

{
  "url": "http://httpbin.org/anything/1",
  "code": 200,
  "body": null
}


In [26]:
# JSON escape the response body
echo "http://httpbin.org/anything/1" |\
  ganda -s -J -B escaped |\
  jq '.'

{
  "url": "http://httpbin.org/anything/1",
  "code": 200,
  "body": "{\n  \"args\": {}, \n  \"data\": \"\", \n  \"files\": {}, \n  \"form\": {}, \n  \"headers\": {\n    \"Accept-Encoding\": \"gzip\", \n    \"Host\": \"httpbin.org\", \n    \"User-Agent\": \"Go-http-client/1.1\", \n    \"X-Amzn-Trace-Id\": \"Root=1-66a6f80a-645548683175583438c5aad2\"\n  }, \n  \"json\": null, \n  \"method\": \"GET\", \n  \"origin\": \"173.16.32.166\", \n  \"url\": \"http://httpbin.org/anything/1\"\n}\n"
}


In [27]:
# calculate the sha256 hash of the response body
echo "http://httpbin.org/anything/1" |\
  ganda -s -J -B sha256 |\
  jq '.'

{
  "url": "http://httpbin.org/anything/1",
  "code": 200,
  "body": "4d9ab7ca3c2f4df9cb810a72d99a4fb2c2073151b5872ed5f4ce0d89c21cbc53"
}


## Customizing Requests with JSON Request Syntax

`ganda` supports an alternate JSON-lines syntax for requests.  The [JSON schema](../request.schema.json) is available, but the summary of fields it allows is:
- `"url"` - required string - is the only required field - the request URL
- `"method"` - optional string - a valid HTTP method (`GET|PUT|POST|DELETE|...`) - defaults to `GET`
- `"headers"` - optional JSON object - string key/value pairs
- `"context"` - optional JSON value - carried forward into the JSON output of the response, used to correlate requests and responses, can be a string, array, or object
- `"body"` - optional JSON value - a string or valid JSON object, the body of the request
- `"bodyType"` - optional enum - one of: `json` (default), `escaped`, or `base64`

### Adding a Request Body

What if you want to `POST` instead of `GET`?  `ganda` supports the same `-X <http method>` syntax that `curl` uses:

In [28]:
seq 3 |\
  awk '{printf "http://httpbin.org/anything/%s\n", $1}' |\
  ganda -s -X POST |\
  jq -c '{method, url}'

{"method":"POST","url":"http://httpbin.org/anything/1"}
{"method":"POST","url":"http://httpbin.org/anything/2"}
{"method":"POST","url":"http://httpbin.org/anything/3"}


But, along with most `POST` requests, you'll want to include a body.  `ganda` has an alternate JSON-lines syntax for requests that allows specifying the method and body:


In [29]:
echo '{ "method": "POST", "url": "http://httpbin.org/anything/1", "body": { "key1": "value1", "key2": "value2" } }' |\
  ganda -s |\
  jq '.'

{
  "args": {},
  "data": "{ \"key1\": \"value1\", \"key2\": \"value2\" }",
  "files": {},
  "form": {},
  "headers": {
    "Accept-Encoding": "gzip",
    "Content-Length": "38",
    "Host": "httpbin.org",
    "User-Agent": "Go-http-client/1.1",
    "X-Amzn-Trace-Id": "Root=1-66a6f80f-029b362b46e4c05b75d3cb4c"
  },
  "json": {
    "key1": "value1",
    "key2": "value2"
  },
  "method": "POST",
  "origin": "173.16.32.166",
  "url": "http://httpbin.org/anything/1"
}


By default, it assumes that the body in the request JSON is also a valid JSON object.  If you've got escaped JSON or base64-encoded binary data, you can use the optional `"bodyType"` JSON field to tell `ganda` to transform your input before sending it.

In [32]:
# "bodyType": "escaped" - ganda will unescape before making the request
echo '{ "method": "POST", "url": "http://httpbin.org/anything/1", "body": "{ \"key1\": \"value1\" }", "bodyType": "escaped" }' |\
  ganda -s |\
  jq '.'

{
  "args": {},
  "data": "{ \"key1\": \"value1\" }",
  "files": {},
  "form": {},
  "headers": {
    "Accept-Encoding": "gzip",
    "Host": "httpbin.org",
    "Transfer-Encoding": "chunked",
    "User-Agent": "Go-http-client/1.1",
    "X-Amzn-Trace-Id": "Root=1-66a6f829-0d39895644d662a87f2d7223"
  },
  "json": {
    "key1": "value1"
  },
  "method": "POST",
  "origin": "173.16.32.166",
  "url": "http://httpbin.org/anything/1"
}


In [33]:
# "bodyType": "base64" - ganda will decode before making the request
# generated with: echo -n '{ "value": "was base64 escaped" }' | base64
echo '{ "method": "POST", "url": "http://httpbin.org/anything/1", "body": "eyAidmFsdWUiOiAid2FzIGJhc2U2NCBlc2NhcGVkIiB9", "bodyType": "base64" }' |\
  ganda -s |\
  jq '.'

{
  "args": {},
  "data": "{ \"value\": \"was base64 escaped\" }",
  "files": {},
  "form": {},
  "headers": {
    "Accept-Encoding": "gzip",
    "Host": "httpbin.org",
    "Transfer-Encoding": "chunked",
    "User-Agent": "Go-http-client/1.1",
    "X-Amzn-Trace-Id": "Root=1-66a6f82b-6fa16df9227feda5346567d7"
  },
  "json": {
    "value": "was base64 escaped"
  },
  "method": "POST",
  "origin": "173.16.32.166",
  "url": "http://httpbin.org/anything/1"
}


### Request Headers

`ganda` allows adding static headers to every request with the `-H key:value` syntax.  Multiple headers can be specified, and they'll override defaulted values:

In [34]:
echo '{ "url": "http://httpbin.org/anything/1" }' |\
  ganda -s -H "X-My-Header: 1234" -H "User-Agent: static-ganda" |\
  jq '.'

{
  "args": {},
  "data": "",
  "files": {},
  "form": {},
  "headers": {
    "Accept-Encoding": "gzip",
    "Host": "httpbin.org",
    "User-Agent": "static-ganda",
    "X-Amzn-Trace-Id": "Root=1-66a6f82c-6852021d63ad8b7367bbabdb",
    "X-My-Header": "1234"
  },
  "json": null,
  "method": "GET",
  "origin": "173.16.32.166",
  "url": "http://httpbin.org/anything/1"
}


The JSON-lines syntax can also specify per-request headers that will override static headers

In [35]:
echo '{ "url": "http://httpbin.org/anything/1", "headers": {"X-Second-Header": "5678", "User-Agent": "per-request-ganda" } }' |\
  ganda -s -H "X-My-Header: 1234" -H "User-Agent: static-ganda" |\
  jq '.'

{
  "args": {},
  "data": "",
  "files": {},
  "form": {},
  "headers": {
    "Accept-Encoding": "gzip",
    "Host": "httpbin.org",
    "User-Agent": "per-request-ganda",
    "X-Amzn-Trace-Id": "Root=1-66a6f82e-0f4a85a04c304ca8314a54a3",
    "X-My-Header": "1234",
    "X-Second-Header": "5678"
  },
  "json": null,
  "method": "GET",
  "origin": "173.16.32.166",
  "url": "http://httpbin.org/anything/1"
}


## Request Context

Your requests are part of a pipeline, what if you want to carry context through your pipeline that isn't part of the HTTP request/response?

An example would be calling an HTTP endpoint to generate a new UUID, but the response does not include the ID that we want to associate with the UUID.

`ganda` allows you to specify values along with the URL that will still be present in the JSON envelope output.

This can be done with the simple request syntax by specifying tab-separated values after the URL:


In [36]:
# echo can emit tab separated values for correlating requests and responses, here is what is being passed to ganda:
echo -e 'http://httpbin.org/uuid\t1\t"single\tvalue\twith\ttabs"'

http://httpbin.org/uuid	1	"single	value	with	tabs"


In [37]:
echo -e 'http://httpbin.org/uuid\t1\t"single\tvalue\twith\ttabs"' |\
  ganda -s -J |\
  jq '.'

{
  "url": "http://httpbin.org/uuid",
  "code": 200,
  "body": {
    "uuid": "c8eb971d-f96f-4c34-92b3-47477ecb2fb4"
  },
  "context": [
    "1",
    "single\tvalue\twith\ttabs"
  ]
}


Notice the `"contex"` emitted at the bottom of the JSON.

The JSON-lines request format also allows context to be specified, and it can be any valid JSON object (string, array, or object):

In [38]:
echo '{ "url": "http://httpbin.org/uuid", "context": { "id": 1, "value": "correlation value"} }' |\
  ganda -s -J |\
  jq '.'

{
  "url": "http://httpbin.org/uuid",
  "code": 200,
  "body": {
    "uuid": "7fe27e6c-e842-46e1-9eae-49cb9d25dcb4"
  },
  "context": {
    "id": 1,
    "value": "correlation value"
  }
}


## `ganda echoserver` - a simple server that echoes requests 

`ganda` comes with a built-in echo server to make verifying requests easier. 

We don't want to hammer the public `httpbin.org` server, so let's fire up `ganda echoserver` as a background process and use that instead. 

In [39]:
ganda echoserver --help

NAME:
   ganda echoserver - Starts an echo server, --port <port> to override the default port of 8080

USAGE:
   ganda echoserver [command [command options]] 

OPTIONS:
   --port value          Port number to start the echo server on (default: 8080)
   --delay-millis value  Number of milliseconds to delay responding (default: 0)
   --help, -h            show help (default: false)


In [3]:
# run the echoserver in the background. give it a 1000ms/1s delay for responding to each request.  
# If you're running it in a separate terminal, you can omit the `>/dev/null &` part
ganda echoserver --port 9090 --delay-millis 1000 >/dev/null &
ECHOSERVER_PID=$! # capture the PID of the echoserver so we can shut it down later

kill_echoserver() {
  kill $ECHOSERVER_PID
}

echo $ECHOSERVER_PID

[1] 57220
57220


Let's use `ganda` to make a single request to our echoserver so we can see its output and that it takes about a second:

In [51]:
time echo "http://localhost:9090/anything/1" | ganda -s | jq '.'

{
  "time": "2024-07-28T21:10:44-05:00",
  "id": "",
  "remote_ip": "::1",
  "host": "localhost:9090",
  "method": "GET",
  "uri": "/anything/1",
  "user_agent": "Go-http-client/1.1",
  "status": 200,
  "headers": {
    "Accept-Encoding": "gzip",
    "Connection": "keep-alive",
    "User-Agent": "Go-http-client/1.1"
  },
  "request_body": ""
}

real	0m1.014s
user	0m0.019s
sys	0m0.008s


## Parallelizing Requests

By default, `ganda` using a single worker thread and a single connection to make requests.  This will guarantee that requests are made in the order they are received. 

If order doesn't matter, and you'd like to increase throughput, we can use the `-W <number of workers>` command.

In [52]:
# our echoserver is running with a 1000ms delay, so this should take about 10 seconds to complete
time seq 10 |\
  awk '{printf "http://localhost:9090/slow-api\n", $1}' |\
  ganda -s > /dev/null


real	0m10.042s
user	0m0.012s
sys	0m0.016s


If we increase the number of workers to 10, we should finish in about a second

In [1]:
# our echoserver is running with a 1 second delay so the 10 requests should be handled by 10 works in about 1 second
time seq 10 |\
  awk '{printf "http://localhost:9090/slow-api\n", $1}' |\
  ganda -s -W 10 > /dev/null


real	0m0.016s
user	0m0.007s
sys	0m0.011s


In [4]:
# if we use 100 parallel workers and make 1k requests that each take 1 second, it should take about 10 seconds
time seq 1000 |\
  awk '{printf "http://localhost:9090/slow-api\n", $1}' |\
  ganda -s -W 100 | 
  pv -albert > /dev/null

1.00k 0:00:10 [99.3 /s] [99.3 /s]

real	0m10.079s
user	0m1.129s
sys	0m0.498s


`ganda` also supports throttling the number of requests its workers will make using the `--throttle <requests per second>` flag.

If we use 100 parallel workers, but throttle them so that they can only make 5 requests per second, it should take about 20 seconds to complete.

In [1]:
time seq 10 |\
  awk '{printf "http://localhost:9090/slow-api\n", $1}' |\
  ganda -W 1 --throttle 1 |\
  pv -albert > /dev/null

Response: 200 http://localhost:9090/slow-api
Response: 200 http://localhost:9090/slow-api
Response: 200 http://localhost:9090/slow-api
Response: 200 http://localhost:9090/slow-api
Response: 200 http://localhost:9090/slow-api
Response: 200 http://localhost:9090/slow-api
Response: 200 http://localhost:9090/slow-api
Response: 200 http://localhost:9090/slow-api
Response: 200 http://localhost:9090/slow-api
Response: 200 http://localhost:9090/slow-api
10.0  0:00:10 [ 971m/s] [ 971m/s]

real	0m10.302s
user	0m0.020s
sys	0m0.037s


In [7]:
# clean up the echoserver that we'd previously run in the background
kill_echoserver && echo "echoserver stopped" || echo "echoserver not stopped"

bash: kill_echoserver: command not found
echoserver not stopped
